In [ ]:
# analise_financeira.ipynb

import pandas as pd
import matplotlib.pyplot as plt

# Carregar os dados
movimentacoes_df = pd.read_csv('../data/extracted/movimentacoes.csv')
movimentacoes_item_df = pd.read_csv('../data/extracted/movimentacoes_itens.csv')
produtos_df = pd.read_csv('../data/extracted/produtos.csv')

# Mesclando dados de movimentações com itens de movimentação
movimentacoes_merged_df = movimentacoes_item_df.merge(movimentacoes_df, left_on='movimentacao_id', right_on='id')
movimentacoes_merged_df = movimentacoes_merged_df.merge(produtos_df, left_on='produto_id', right_on='id', suffixes=('_mov', '_prod'))

# Calculando receita e lucro por período
movimentacoes_merged_df['criado_em'] = pd.to_datetime(movimentacoes_merged_df['criado_em'])
movimentacoes_merged_df['month'] = movimentacoes_merged_df['criado_em'].dt.to_period('M')
movimentacoes_merged_df['lucro'] = movimentacoes_merged_df['preco_venda'] - movimentacoes_merged_df['preco_custo']

monthly_finance = movimentacoes_merged_df.groupby('month').agg({
    'preco_venda': 'sum',
    'preco_custo': 'sum',
    'lucro': 'sum'
}).reset_index()

# Plotar análise financeira
plt.figure(figsize=(10, 6))
plt.plot(monthly_finance['month'].astype(str), monthly_finance['preco_venda'], marker='o', label='Receita')
plt.plot(monthly_finance['month'].astype(str), monthly_finance['preco_custo'], marker='o', label='Custo')
plt.plot(monthly_finance['month'].astype(str), monthly_finance['lucro'], marker='o', label='Lucro')
plt.title('Análise Financeira Mensal')
plt.xlabel('Mês')
plt.ylabel('Valor')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# Salvar resultados em CSV
monthly_finance.to_csv('../data/processed/analise_financeira.csv', index=False)
